# Generadores y Corrutinas (Generators and Corroutines)

## Tabla de contenidos
***
- [Como crear generadores](#id1)
<a href='#id1'></a>

- [Generadores infinitos](#id2)
<a href='#id2'></a>

- [Generadores envolviendo iterables](#id3)
<a href='#id3'></a>

- [Comprehensions de generadores](#id4)
<a href='#id4'></a>

- [Generadores envolviendo iterables](#id5)
<a href='#id5'></a>

- [Comprehensions de generadores](#id6)
<a href='#id6'></a>

- [Generadores e iteradores basados en clases](#id7)
<a href='#id7'></a>

- [itertools.islice](#id8)
<a href='#id8'></a>

- [itertools.chain](#id9)
<a href='#id9'></a>

- [itertools.tee](#id10)
<a href='#id10'></a>

- [contextlib.contextmanager - Creando context managers](#id11)
<a href='#id11'></a>

- [Corrutinas](#id12)
<a href='#id12'></a>

- [Priming](#id13)
<a href='#id13'></a>


***

Los generadores son funciones que se comportan como iteradores generando los valores que retornan uno a uno.

Un generador ocupa `yield` para retornar un único valor cuando se le es solicitado. El efecto lateral es que los generadores pueden ser infinitamente largos porque se puede seguir solicitando por siempre.

Hay una variación en la sintaxís de los generadores que crean corrutinas. Las corrutinas son funciones que permiten el multitasking sin requerir múltiples threads o procesos. 

Mientras que los generadores pueden retornar values a quien se lo solicita basado en los argumentos iniciales, las corrutinas permiten dos formas de comunicación con la función que la llama mientras corre. La implementación de corrutinas en Python es a través del módulo `asyncio`.

Si las corrutinas funcionan para lo que usted necesita, pueden ofrecer una enorme mejora en el rendimiento.

## Generadores

A continuación se presentan las principales ventajas y desventajas de los generadores.

**Ventajas**

- Los generadores son usualmente más simples que escribir que una función que genera listas. Sólo se necesita el valor `yield`.
- Los ítems pueden ser procesados uno a la vez, así que generalmente no hay necesidad de mantener toda la lista en la memoria.
- Los resultados pueden depender de factores externos. Sólo se genera el valor cuando es solicitado.
- Los generadores son "flojos". Esto quiere decir, que si se usan solo los diez primeros resultados de un generador, el resto no va a ser calculado.

**Desventajas**
- Los resultados sólo están disponibles una vez. Después de procesar los resultados de un generador, no se pueden usar de nuevo.
- El tamaño es desconocido, incluso puede ser infinito.
- No es posible usar slices, así que mi_generador[3:10] no va a funcionar. Se puede usar itertools.islice para esto.
- No se puede indexar generadores, así que mi_generador[3] no funcionará.

<a id='id1'></a>

## Como crear generadores

El generador más simple que se puede crear es una función que contiene una declaración `yield` en vez de `return`. La diferencia principal con funciones regulares que contienen un `return`, es que se pueden tener muchos `yield` en la función.

In [1]:
def mi_generador():
    yield 5
    yield "Buenos días"
    yield 25
    return "Hemos llegado al final"

resultado = mi_generador()

In [2]:
resultado

<generator object mi_generador at 0x000001F8332A7890>

In [3]:
len(resultado)

TypeError: object of type 'generator' has no len()

In [4]:
resultado[:]

TypeError: 'generator' object is not subscriptable

In [5]:
list(resultado)

[5, 'Buenos días', 25]

In [6]:
list(resultado)

[]

Como se puede observar, no se obtiene mucha información valiosa al con `repr()`, `len()` o slices. Además, utilizar `list()` una segunda oportunidad no funciona porque el generador ya está "cansado".

Además, puede que haya notado que el valor de `return` pareciera que ha desaparecido. No es el caso, el valor de return todavía se puede utilizar, pero como el valor para la excepción `StopIteration` producidad por el generador para indicar que está "exhausto".

In [7]:
def segundo_generador():
    yield "Algún valor"
    return "El fin del generador"

resultado = segundo_generador()

In [8]:
next(resultado)

'Algún valor'

In [9]:
next(resultado)

StopIteration: El fin del generador

Mencionar que nosotros podemos manejar la excepción, de `StopIteration`, de la siguiente manera (por ejemplo).

In [10]:
def flojo():
    print("Antes del yield statement")
    yield "He llegado a yield"
    print("Después del yield statement")
    
generador = flojo()

In [11]:
next(generador)

Antes del yield statement


'He llegado a yield'

In [12]:
try:
    next(generador)
except StopIteration:
    print("He llegado al final, pues se ha producido una excepción del tipo StopIteration")

Después del yield statement
He llegado al final, pues se ha producido una excepción del tipo StopIteration


In [13]:
for item in flojo():
    print(item)

Antes del yield statement
He llegado a yield
Después del yield statement


Para el correcto manejo de los generadores, siempre tiene que manejar usted la excepción `StopIteration`, o usar un loop u otra estructura que maneje `StopIteration` de forma implícita.

<a id='id2'></a>

## Creando generadores infinitos

Crear un generador sin fin es sencillo. En vez de tener `yield` <valor> como en las funciones previas, ocupamos `yield` dentro de un loop infinito, así podemos hacer un generador infinito.

In [14]:
def contar(start=0, steps=1, stop= None):
    n = start
   
    while stop is not None and n < stop:
        yield n
        n += steps
        
list(contar(5, 1, 10))

[5, 6, 7, 8, 9]

**Debido a la naturaleza infinita de los generadores, se requiere precaución. Sin la variable stop, hacer list(contar()) puede resultar en un loop infinito que puede resultar en quedarse sin memoria de forma sumamente rápida.**

Lo anterior es en esencia, un for loop de toda la vida, la principal diferencia entre esto y el método regular de retornar una lista de items is que el `yield` statement retorna un item a la vez, lo que significa que solo se necesita calcular el ítem solicitado y no se deben de mantener todos los resultados en la memoria.

<a id='id5'></a>

## Generadores envolviendo (wrapping) iterables

El verdadero poder de los generadores se produce cuando se utilizan con otros iterables.

In [15]:
def cuadrado(iterable):
    for i in iterable:
        yield i ** 2
        
list(cuadrado(range(6)))

[0, 1, 4, 9, 16, 25]

Debido a que los generadores son iterables, se pueden combinar "envolviéndolos" tantas veces como usted desee.

In [16]:
def impar(iterable):
    for i in iterable:
        if i % 2:
            yield i
            
def cuadrado(iterable):
    for i in iterable:
        yield i ** 2
        

list(cuadrado(impar(range(10))))

[1, 9, 25, 49, 81]

Para entender lo que sucedió, debemos ir desde el interior hasta el exterior.
- `range(10)` genera 10 números
- El generador `impar()` filtra los valores, así solo retorna los números impares.
- El generador `cuadrado()` eleva a la potencia de dos los números impares.

El verdadero poder de combinar es que los generadores harán algo únicamente cuando se les pida que lo hagan. Si solicitamos un solo valor con `next()` en vez de `list()`, solo la primera iteración en el loop en `cuadrado()` será ejecutada.

<a id='id6'></a>

## Comprehensions de generadores

La premisa básica es idéntica a las list comprehensions, pero se usan paréntesis en vez de paréntesis cuadrados.

In [17]:
cuadrados = (x ** 2 for x in range(5))

In [18]:
cuadrados

<generator object <genexpr> at 0x000001F834E56890>

In [19]:
list(cuadrados)

[0, 1, 4, 9, 16]

Lo anterior es sumamente útil cuando se necesitan "envolver" los resultados de un generador distinto, pues solo calcula los valores que se le pide.

In [20]:
import itertools

resultado = itertools.count()
impar = (x for x in resultado if x % 2)
slices_impares = itertools.islice(impar, 5)
list(slices_impares)

[1, 3, 5, 7, 9]

In [21]:
import itertools

resultado = itertools.count()
slice_resultados = itertools.islice(resultado, 5)
impar = (x for x in slice_resultados if x % 2)
list(impar)

[1, 3]

Como puede notar, el orden de las operaciones es sumamente importate, pues la función itertools.islice() hace slice al resultado en tal punto, no el generador original.

<a id='id7'></a>

## Generadores e iteradores basados en clases

El crear generadores utilizando clases, puede ser beneficioso para generadores más complejos donde se requiere recordar el estado o donde se puede usar herencia.

A continuación se creará una clase que intenta imitar el comportamiento de `itertools.count()` con un parámetro de stop.

In [22]:
class ContadorGenerator:
    def __init__(self, start=0, step=1, stop=None):
        self.start = start
        self.step = step
        self.stop = stop
    
    
    def __iter__(self):
        i = self.start
        
        while self.stop is None or i < self.stop:
            yield i
            i += self.step
            

In [23]:
list(ContadorGenerator(start=2.5, step=0.5, stop=5))

[2.5, 3.0, 3.5, 4.0, 4.5]

A continuación se modificará un poco la clase generador para convertirla en un iterador con más características.

In [24]:
class ContadorIterador:
    def __init__(self, start = 0, step = 1, stop = None):
        self.i = start
        self.start = start
        self.step = step
        self.stop = stop
    
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.stop is not None and self.i >= self.stop:
            raise StopIteration
        
        #Se requiere retornar el valor antes de incrementarlo
        value = self.i
        self.i += self.step
        return value

In [25]:
list(ContadorIterador(start = 2.5, step = 0.5, stop = 5))

[2.5, 3.0, 3.5, 4.0, 4.5]

La distinción más importante entre el generador y el iterador, es que en vez de tener un simple objecto iterador, ahora tenemos una clase más desarrollada que actúa como un iterador, lo que significa que podemos extender sus capacidades más allá de los generadores regulares.

Una de las cuantas limitaciones de generadores regulares es que no se puede usar `len()` ni se puede usar slices.

In [26]:
import itertools

class IteradorAvanzado:
    
    def __init__(self, start = 0, step = 1, stop = None):
        self.i = start
        self.start = start
        self.step = step
        self.stop = stop
    
    
    def __iter__(self):
        return self
    
    
    def __next__(self):
        if self.stop is not None and self.i >= self.stop:
            raise StopIteration
            
        value = self.i
        self.i += self.step
        return value
    
    
    def __len__(self):
        return int((self.stop - self.start) // self.step)

    
    def __contains__(self, key):
        return self.start < key < self.stop
    
    
    def __repr__(self):
        return (
            f"{self.__class__.__name__}(start = {self.start}),"
            f"step = {self.step}, stop = {self.stop}"
        )
    
    
    def __getitem__(self, slice_):
        return itertools.islice(self, slice_.start, slice_.stop, slice_.step)

Ahora tenemos un iterador que soporta `len()`, `in` y `repr()`.

In [27]:
count = IteradorAvanzado(start=3, step=0.5, stop=7)

In [28]:
count

IteradorAvanzado(start = 3),step = 0.5, stop = 7

In [29]:
4 in count

True

In [30]:
1 in count

False

In [31]:
len(count)

8

In [32]:
count[:3]

In [33]:
list(count[:3])

[3, 3.5, 4.0]

Antes de que empiece a programar generadores para su proyecto, revise primero el módulo `itertools` de Python. Contiene muchos generadores muy útiles que abarcan una amplia cantidad de casos.

<a id='id8'></a>

## itertools.islice - Slicing de iterables

Una limitación de los generadores es que no se puede usar slicing. Se puede "resolver" convirtiendo el generador en una lista antes de usar slicing, pero lo anterior no es posible con generadores infinitos, y lo anterior puede ser ineficiente si solo se necesitan un par de valores.

Para resolver esto, la librería `itertools` tiene una función `islice()`, que puede hacer slicing en cualquier objeto iterable. La función soporta los parámetros start, stop, y step.

In [34]:
import itertools

lista = list(range(1000))
lista[:5]

[0, 1, 2, 3, 4]

In [35]:
list(itertools.islice(lista,5))

[0, 1, 2, 3, 4]

Si bien el output es idéntico, el funcionamiento interno de los métodos difiere notablemente. En algunos casos `itertools.islice()` puede ser más lento que simplemente usar [a:b], lo único que para usar esto, se requieren objetos que soporten slicing.

<a id='id9'></a>

## itertools.chain - Concatenando múltiples iterables

Esto retorna cada item de cada iterable en orden secuencial.

In [36]:
def chain(*iterables):
    for iterable in iterables:
        yield from iterable

In [37]:
a = 1,2,3
b = [4,5,6]
c = 'abc'

list(chain(a,b,c))

[1, 2, 3, 4, 5, 6, 'a', 'b', 'c']

In [38]:
a + b + c

TypeError: can only concatenate tuple (not "list") to tuple

La expresión `yield from` "retorna" todos los items del iterable.

<a id='id10'></a>

## itertools.tee - Usando un output múltiples ocasiones

Este método retorna múltiples iteradores, permitiendo que puedan ser procesados de formas separadas.

Por defecto, `tee` va a separar el generador en una tupla que contiene dos generadores distintos, lo que significa que tuple unpacking funciona de buena manera aquí.

In [39]:
import itertools

def fizz_and_buzz():
    yield 'fizz'
    yield 'buzz'

In [40]:
a, b = itertools.tee(fizz_and_buzz())

In [41]:
next(a)

'fizz'

In [42]:
next(a)

'buzz'

In [43]:
next(b)

'fizz'

In [44]:
next(b)

'buzz'

In [45]:
next(b)

StopIteration: 

<a id='id11'></a>

## contextlib.contextmanager - Creando context managers

Si bien contextlib.contextmanager() es un generador, no fue pensado para ser un generador de resultados, pero usa `yield`, lo que lo convierte en un buen ejemplo de uso no estándar.

In [46]:
import time
import datetime
import contextlib

@contextlib.contextmanager
def timer(name):
    start_time = datetime.datetime.now()
    yield
    stop_time = datetime.datetime.now()
    print('%s took %s' % (name, stop_time - start_time))
    
    
    
@contextlib.contextmanager
def write_to_log(name):
    with contextlib.ExitStack() as stack:
        fh = stack.enter_context(open(f'{name}.txt', 'w'))
        stack.enter_context(contextlib.redirect_stdout(fh))
        stack.enter_context(timer(name))
        yield
        
        
@write_to_log('some_name')
def some_function():
    print('This will be written to `some_name.txt`')

In [47]:
some_function()

<a id='id12'></a>

## Corrutinas

La premisa básica is que las corrutinas permite la comunicación entre dos funciones, mientras cada una de ellas corre en un único thread.

Con corrutinas la comunicación ocurre de dos maneras; la corrutina puede recibir valores, al igual que `yield` estos a la función que la llama.

Para objetivos prácticos, se recomienda usar `asyncio` en vez de la syntaxís de las corrutinas. Para objetivos educacionesl, es útil entender como funcionan.

In [48]:
def generador():
    valor = yield 'valor de generador'
    print(f"El generador recibió {valor}")
    yield f"Valor previo: {valor!r}"

In [49]:
g = generador()
print(f"Resultado del generador: {next(g)}")

Resultado del generador: valor de generador


In [50]:
print(g.send("Valor de quien llama"))

El generador recibió Valor de quien llama
Valor previo: 'Valor de quien llama'


La función se mantiene inactiva hasta que el método `send` es utilizado. El intercambio de valores solo puede suceder cuando el código corre `next(generator)` o `generator.send()`

<a id='id13'></a>

## Priming

Antes de que un valor pueda ser enviado al generador, un resultado debe ser buscado usando `next()` o `send(None)` debe ser utilizado, para que así el código sea alcanzado.